In [42]:
import simple_data_analysis.config
import simple_data_analysis.db as db
import tabulate as tabulate


def write_output(result: list):
    
    if not result:
        print(tabulate.tabulate([['NO DATA']], headers=['OOPPPPSSS'], tablefmt='outline', stralign="left", numalign="center"))
        return
    
    keys = list(result[0].keys())
    output = []
    
    for row in result:
        output.append([row[key] for key in keys])

    print(tabulate.tabulate(output, headers=keys, tablefmt='outline', stralign="left", numalign="center"))

# Question 1

<p style="direction: rtl; text-align: left">10 آهنگ برتر که بیشترین درامد رو داشتن به همراه درامد ایجاد شده</p>

In [43]:
sql = '''
SELECT i.TrackId, t.Name, SUM(i.UnitPrice * i.Quantity) AS total_price
FROM InvoiceLine AS i
JOIN Track AS t ON t.TrackId=i.TrackId
GROUP BY i.TrackId
ORDER BY total_price DESC LIMIT 10
'''

db.db.execute(sql)
write_output(db.db.fetchall())

+-----------+------------------------------+---------------+
|  TrackId  | Name                         |  total_price  |
+===========+==============================+===============+
|   2832    | The Woman King               |     3.98      |
|   3223    | How to Stop an Exploding Man |     3.98      |
|   2850    | The Fix                      |     3.98      |
|   3250    | Pilot                        |     3.98      |
|   3214    | Phyllis's Wedding            |     3.98      |
|   3200    | Gay Witch Hunt               |     3.98      |
|   2868    | Walkabout                    |     3.98      |
|   3177    | Hot Girl                     |     3.98      |
|   2844    | Better Halves                |     1.99      |
|   2821    | Exodus, Pt. 1                |     1.99      |
+-----------+------------------------------+---------------+


# Question 2
<p style="direction: rtl; text-align: left">محبوب ترین ژانر، به ترتیب از نظر تعداد آهنگ های فروخته شده و کل درامد</p>

In [44]:
sql = '''
SELECT g.GenreId, g.Name, SUM(i.UnitPrice * i.Quantity) AS total_price, COUNT(i.TrackId) AS total_sell
FROM InvoiceLine AS i
JOIN Track AS t ON t.TrackId=i.TrackId
JOIN Genre AS g ON g.GenreId=t.GenreId
GROUP BY t.GenreId
ORDER BY total_price DESC, total_sell DESC
LIMIT 10
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+-----------+--------------------+---------------+--------------+
|  GenreId  | Name               |  total_price  |  total_sell  |
+===========+====================+===============+==============+
|     1     | Rock               |    826.65     |     835      |
|     7     | Latin              |    382.14     |     386      |
|     3     | Metal              |    261.36     |     264      |
|     4     | Alternative & Punk |    241.56     |     244      |
|    19     | TV Shows           |     93.53     |      47      |
|     2     | Jazz               |     79.2      |      80      |
|     6     | Blues              |     60.39     |      61      |
|    21     | Drama              |     57.71     |      29      |
|    14     | R&B/Soul           |     40.59     |      41      |
|    24     | Classical          |     40.59     |      41      |
+-----------+--------------------+---------------+--------------+


# Question 3
<p style="direction: rtl; text-align: left">کاربرانی که تا حالا خرید نداشتند</p>

In [45]:
sql = '''
SELECT *
FROM Customer AS c
WHERE c.CustomerId NOT IN(SELECT CustomerId FROM Invoice GROUP BY CustomerId)
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+-------------+
| OOPPPPSSS   |
+=============+
| NO DATA     |
+-------------+


# Question 4
<p style="direction: rtl; text-align: left">میانگین زمان آهنگ ها در هر آلبوم</p>

In [46]:
sql = '''
SELECT a.AlbumId, a.Title, AVG(t.Milliseconds) AS millisecond_avg
FROM Track AS t
JOIN Album AS a ON a.AlbumId=t.AlbumId
GROUP BY t.AlbumId
ORDER BY millisecond_avg DESC
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+-----------+-------------------------------------------------------------------------------------------------+-------------------+
|  AlbumId  | Title                                                                                           |  millisecond_avg  |
+===========+=================================================================================================+===================+
|    253    | Battlestar Galactica (Classic), Season 1                                                        |    2.92557e+06    |
|    227    | Battlestar Galactica, Season 3                                                                  |    2.77827e+06    |
|    229    | Lost, Season 3                                                                                  |    2.71791e+06    |
|    231    | Lost, Season 2                                                                                  |    2.63707e+06    |
|    226    | Battlestar Galactica: The Story So Far                        

# Question 5
<p style="direction: rtl; text-align: left">کارمندی که بیشترین تعداد فروش را داشته است</p>

In [47]:
sql = '''
SELECT e.EmployeeId, CONCAT(e.FirstName, " ", e.LastName) AS fullname, COUNT(c.CustomerId) AS total_sell
FROM Invoice AS i
JOIN Customer AS c ON c.CustomerId=i.CustomerId
JOIN Employee AS e ON e.EmployeeId=c.SupportRepId
GROUP BY c.SupportRepId
ORDER BY total_sell DESC
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+--------------+---------------+--------------+
|  EmployeeId  | fullname      |  total_sell  |
+==============+===============+==============+
|      3       | Jane Peacock  |     146      |
|      4       | Margaret Park |     140      |
|      5       | Steve Johnson |     126      |
+--------------+---------------+--------------+


# Question 6
<p style="direction: rtl; text-align: left">کاربرانی که بیش از یک ژانر خرید کردند</p>

In [48]:
sql = '''
SELECT o.CustomerId, CONCAT(c.FirstName, " ", c.LastName) AS customer_fullname, COUNT(o.CustomerId) AS genre_count
FROM (
    SELECT i.CustomerId, t.GenreId
    FROM Invoice AS i
    JOIN InvoiceLine AS il ON il.InvoiceId=i.InvoiceId
    JOIN Track AS t ON t.TrackId=il.TrackId
    GROUP BY i.CustomerId, t.GenreId
) AS o
JOIN Customer AS c ON c.CustomerId=o.CustomerId
GROUP BY o.CustomerId
HAVING genre_count > 1
ORDER BY genre_count DESC
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+--------------+-----------------------+---------------+
|  CustomerId  | customer_fullname     |  genre_count  |
+==============+=======================+===============+
|      57      | Luis Rojas            |      12       |
|      45      | Ladislav Kovács       |      11       |
|      37      | Fynn Zimmermann       |      10       |
|      14      | Mark Philips          |      10       |
|      17      | Jack Smith            |      10       |
|      3       | François Tremblay     |      10       |
|      34      | João Fernandes        |      10       |
|      24      | Frank Ralston         |      10       |
|      23      | John Gordon           |       9       |
|      41      | Marc Dubois           |       9       |
|      27      | Patrick Gray          |       9       |
|      21      | Kathy Chase           |       9       |
|      7       | Astrid Gruber         |       9       |
|      43      | Isabelle Mercier      |       9       |
|      6       | Helena Holý   

# Question 7
<p style="direction: rtl; text-align: left">سه آهنگ برتر از نظر درآمد فروش برای هر ژانر</p>

In [52]:
sql = '''
SELECT r.TrackId, r.TrackName, r.GenreId, r.GenreName, r.total_price, GenreNo
FROM (
    SELECT t.TrackId, t.Name AS TrackName, t.GenreId, g.Name AS GenreName, SUM(i.Quantity * i.UnitPrice) AS total_price,
    ROW_NUMBER() OVER(PARTITION BY t.GenreId ORDER BY total_price DESC) AS GenreNo
    FROM InvoiceLine AS i
    JOIN Track AS t ON t.TrackId=i.TrackId
    JOIN Genre AS g ON g.GenreId=t.GenreId
    GROUP BY t.GenreId, t.TrackId
    ORDER BY t.GenreId ASC, total_price DESC
) AS r
WHERE r.GenreNo <= 3;
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+-----------+------------------------------------------------------------------------------------------+-----------+--------------------+---------------+-----------+
|  TrackId  | TrackName                                                                                |  GenreId  | GenreName          |  total_price  |  GenreNo  |
+===========+==========================================================================================+===========+====================+===============+===========+
|   1708    | Mr. Cab Driver                                                                           |     1     | Rock               |     1.98      |     1     |
|   2263    | Somebody To Love                                                                         |     1     | Rock               |     1.98      |     2     |
|   1412    | Don't Look To The Eyes Of A Stranger                                                     |     1     | Rock               |     1.98      |     3     |
|   

# Question 8
<p style="direction: rtl; text-align: left">تعداد آهنگ های فروخته شده به صورت تجمعی در هر سال به صورت جداگانه</p>

In [60]:
sql = '''
SELECT years.y, COUNT(il.TrackId) AS TotalTrackCount
FROM(
	SELECT YEAR(InvoiceDate) AS y FROM Invoice GROUP BY y ORDER BY y ASC
) AS years
LEFT JOIN Invoice AS i ON YEAR(i.InvoiceDate) <= years.y
LEFT JOIN InvoiceLine AS il ON il.InvoiceId=i.InvoiceId
GROUP BY years.y;
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+------+-------------------+
|  y   |  TotalTrackCount  |
+======+===================+
| 2021 |        454        |
| 2022 |        909        |
| 2023 |       1351        |
| 2024 |       1798        |
| 2025 |       2240        |
+------+-------------------+


# Question 9
<p style="direction: rtl; text-align: left">کاربرانی که مجموع خریدشان بالاتر از میانگین مجموع خرید تمام کاربران است</p>

In [59]:
# customers average price against all customers average price
sql = '''
SELECT i.CustomerId, CONCAT(c.FirstName, " ", c.LastName) AS CustomerFullname, AVG(il.UnitPrice * il.Quantity) AS CustomerAvgPrice
FROM InvoiceLine AS il
JOIN Invoice AS i ON i.InvoiceId=il.InvoiceId
JOIN Customer AS c ON c.CustomerId=i.CustomerId
GROUP BY i.CustomerId
HAVING CustomerAvgPrice > (SELECT AVG(UnitPrice * Quantity) AS TotalAvg FROM InvoiceLine);
'''

# customers total price against all customers average price
sql = '''
SELECT i.CustomerId, CONCAT(c.FirstName, " ", c.LastName) AS CustomerFullname, SUM(il.UnitPrice * il.Quantity) AS CustomerTotalPrice
FROM InvoiceLine AS il
JOIN Invoice AS i ON i.InvoiceId=il.InvoiceId
JOIN Customer AS c ON c.CustomerId=i.CustomerId
GROUP BY i.CustomerId
HAVING CustomerTotalPrice > (SELECT AVG(UnitPrice * Quantity) AS TotalAvg FROM InvoiceLine);
'''
db.db.execute(sql)
write_output(db.db.fetchall())

+--------------+-----------------------+----------------------+
|  CustomerId  | CustomerFullname      |  CustomerTotalPrice  |
+==============+=======================+======================+
|      1       | Luís Gonçalves        |        39.62         |
|      2       | Leonie Köhler         |        37.62         |
|      3       | François Tremblay     |        39.62         |
|      4       | Bjørn Hansen          |        39.62         |
|      5       | František Wichterlová |        40.62         |
|      6       | Helena Holý           |        49.62         |
|      7       | Astrid Gruber         |        42.62         |
|      8       | Daan Peeters          |        37.62         |
|      9       | Kara Nielsen          |        37.62         |
|      10      | Eduardo Martins       |        37.62         |
|      11      | Alexandre Rocha       |        37.62         |
|      12      | Roberto Almeida       |        37.62         |
|      13      | Fernanda Ramos        |